In [1]:
import cv2
import mediapipe as mp
import numpy as np
import json
import os
from tqdm import tqdm
from math import sqrt

# --- Configuration (Centralized Settings) ---
config = {
    "VIDEO_SOURCE_DIR": r"D:\FYP\Dataset\wlasl-complete\videos",
    "SPLIT_FILE_PATH": r"D:\WLASL DataSet\WLASL FULL\nslt_300.json",
    "OUTPUT_NPZ_PATH": r"D:\Landmarks\Bones+Joints\Landmarks_GCN_augmented.npz",
    "SAVE_CHECKPOINT_EVERY_N_VIDEOS": 250,
    "POSE_NODES": [
        mp.solutions.holistic.PoseLandmark.NOSE,
        mp.solutions.holistic.PoseLandmark.LEFT_SHOULDER,
        mp.solutions.holistic.PoseLandmark.RIGHT_SHOULDER,
        mp.solutions.holistic.PoseLandmark.LEFT_ELBOW,
        mp.solutions.holistic.PoseLandmark.RIGHT_ELBOW,
        mp.solutions.holistic.PoseLandmark.LEFT_WRIST,
        mp.solutions.holistic.PoseLandmark.RIGHT_WRIST,
        mp.solutions.holistic.PoseLandmark.LEFT_HIP,
        mp.solutions.holistic.PoseLandmark.RIGHT_HIP,
        mp.solutions.holistic.PoseLandmark.LEFT_KNEE,
        mp.solutions.holistic.PoseLandmark.RIGHT_KNEE,
        mp.solutions.holistic.PoseLandmark.LEFT_ANKLE,
        mp.solutions.holistic.PoseLandmark.RIGHT_ANKLE,
        mp.solutions.holistic.PoseLandmark.LEFT_INDEX,
        mp.solutions.holistic.PoseLandmark.RIGHT_INDEX
    ],
    "FACE_GROUPS": [
        mp.solutions.face_mesh.FACEMESH_LIPS,
        mp.solutions.face_mesh.FACEMESH_LEFT_EYE,
        mp.solutions.face_mesh.FACEMESH_RIGHT_EYE,
        mp.solutions.face_mesh.FACEMESH_LEFT_IRIS if hasattr(mp.solutions.face_mesh, "FACEMESH_LEFT_IRIS") else (),
        mp.solutions.face_mesh.FACEMESH_RIGHT_IRIS if hasattr(mp.solutions.face_mesh, "FACEMESH_RIGHT_IRIS") else (),
    ]
}

# --- Node selection choices ---
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh
mp_hands = mp.solutions.hands

# Utilities
def lm_to_xyzc(lm, default_conf=1.0):
    """Return (x,y,z,confidence) given a mediapipe landmark. Some landmarks have 'visibility' or not."""
    x = lm.x if hasattr(lm, 'x') else 0.0
    y = lm.y if hasattr(lm, 'y') else 0.0
    z = lm.z if hasattr(lm, 'z') else 0.0
    conf = getattr(lm, 'visibility', None)
    if conf is None:
        conf = default_conf
    return [x, y, z, conf]

def sample_face_indices():
    """Take the union of indices from chosen FACE_GROUPS and pick representative indices (~8)."""
    uniq = []
    for group in config["FACE_GROUPS"]:
        if not group:
            continue
        for pair in group:
            if isinstance(pair, (tuple, list)) and len(pair) >= 2:
                for idx in pair[:2]:
                    if idx not in uniq:
                        uniq.append(idx)
    if len(uniq) < 8:
        uniq = list(range(8))
    step = max(1, len(uniq) // 8)
    chosen = uniq[0:8*step:step][:8]
    return chosen

# Precompute face indices we will keep
FACE_INDICES_TO_KEEP = sample_face_indices()

def build_adjacency_map(node_labels):
    """
    node_labels: list of tuples (type, sublabel) to identify each node, e.g.
      ('pose','LEFT_WRIST'), ('pose_mid','mid_shoulder'), ('hand','L_0'), etc.
    Returns adjacency as an (N,N) binary numpy array.
    """
    N = len(node_labels)
    adj = np.zeros((N, N), dtype=np.uint8)
    label_to_idx = {lbl: i for i, lbl in enumerate(node_labels)}

    if hasattr(mp.solutions.pose, "POSE_CONNECTIONS"):
        pose_conn = mp.solutions.pose.POSE_CONNECTIONS
    else:
        pose_conn = ()

    for (a, b) in pose_conn:
        a_name = mp_holistic.PoseLandmark(a).name if isinstance(a, int) else None
        b_name = mp_holistic.PoseLandmark(b).name if isinstance(b, int) else None
        key_a = ('pose', a_name)
        key_b = ('pose', b_name)
        if key_a in label_to_idx and key_b in label_to_idx:
            ia, ib = label_to_idx[key_a], label_to_idx[key_b]
            adj[ia, ib] = 1
            adj[ib, ia] = 1

    pose_node_indices = [label_to_idx.get(('pose', p.name)) for p in config["POSE_NODES"]]
    pose_node_indices = [i for i in pose_node_indices if i is not None]
    for i in range(len(pose_node_indices)-1):
        a, b = pose_node_indices[i], pose_node_indices[i+1]
        adj[a, b] = 1
        adj[b, a] = 1

    for mid_name, endpoints in [('mid_shoulder', ('LEFT_SHOULDER','RIGHT_SHOULDER')),
                                 ('mid_hip', ('LEFT_HIP','RIGHT_HIP'))]:
        mid_key = ('pose_mid', mid_name)
        if mid_key in label_to_idx:
            mid_i = label_to_idx[mid_key]
            for ep in endpoints:
                ep_key = ('pose', ep)
                if ep_key in label_to_idx:
                    ei = label_to_idx[ep_key]
                    adj[mid_i, ei] = 1
                    adj[ei, mid_i] = 1

    if hasattr(mp_hands, "HAND_CONNECTIONS"):
        for hand_side in ('L','R'):
            for (a, b) in mp_hands.HAND_CONNECTIONS:
                node_a = ('hand', f"{hand_side}_{a}")
                node_b = ('hand', f"{hand_side}_{b}")
                if node_a in label_to_idx and node_b in label_to_idx:
                    ia, ib = label_to_idx[node_a], label_to_idx[node_b]
                    adj[ia, ib] = 1
                    adj[ib, ia] = 1

    for side, pose_label in [('L','LEFT_WRIST'), ('R','RIGHT_WRIST')]:
        pose_key = ('pose', pose_label)
        hand_key = ('hand', f"{side}_0")
        if pose_key in label_to_idx and hand_key in label_to_idx:
            pi = label_to_idx[pose_key]
            hi = label_to_idx[hand_key]
            adj[pi, hi] = 1
            adj[hi, pi] = 1

    nose_key = ('pose', 'NOSE')
    if nose_key in label_to_idx:
        ni = label_to_idx[nose_key]
        for fi, lbl in enumerate(node_labels):
            if lbl[0] == 'face':
                fi_idx = label_to_idx[lbl]
                adj[ni, fi_idx] = 1
                adj[fi_idx, ni] = 1

    return adj

def extract_compact_landmarks(video_path, holistic_model):
    """
    Returns a tuple:
    (array (T, num_nodes, 4) -> (x,y,z,conf) or None on failure,
     array (T, 1) -> torso length or None,
     array (T, num_bones, 3) -> bone vectors or None)
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Could not open", video_path)
        return None, None, None

    frames = []
    torso_lengths = []
    bone_vectors_per_frame = []

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic_model.process(image)
            image.flags.writeable = True

            node_features = []
            frame_torso_length = 0.0
            frame_bone_vectors = []

            # --- Extract Pose, Hands, Face Landmarks ---
            if results.pose_landmarks:
                for p in config["POSE_NODES"]:
                    lm = results.pose_landmarks.landmark[p]
                    node_features.append(lm_to_xyzc(lm, default_conf=getattr(lm, 'visibility', 1.0)))
                # midpoints
                left_sh = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER]
                right_sh = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
                mid_sh = [(left_sh.x + right_sh.x)/2, (left_sh.y + right_sh.y)/2, (left_sh.z + right_sh.z)/2, 1.0]
                node_features.append(mid_sh)

                left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP]
                right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP]
                mid_hip = [(left_hip.x + right_hip.x)/2, (left_hip.y + right_hip.y)/2, (left_hip.z + right_hip.z)/2, 1.0]
                node_features.append(mid_hip)
                
                # --- Calculate Torso Length (for normalization) ---
                # We'll use the distance between the mid-shoulder and mid-hip points
                # as a robust measure of torso length.
                dx = mid_sh[0] - mid_hip[0]
                dy = mid_sh[1] - mid_hip[1]
                dz = mid_sh[2] - mid_hip[2]
                frame_torso_length = sqrt(dx**2 + dy**2 + dz**2)
            else:
                for _ in range(len(config["POSE_NODES"]) + 2):
                    node_features.append([0.0, 0.0, 0.0, 0.0])

            # Left hand
            if results.left_hand_landmarks:
                for lm in results.left_hand_landmarks.landmark:
                    node_features.append(lm_to_xyzc(lm, default_conf=1.0))
            else:
                for _ in range(21):
                    node_features.append([0.0, 0.0, 0.0, 0.0])

            # Right hand
            if results.right_hand_landmarks:
                for lm in results.right_hand_landmarks.landmark:
                    node_features.append(lm_to_xyzc(lm, default_conf=1.0))
            else:
                for _ in range(21):
                    node_features.append([0.0, 0.0, 0.0, 0.0])

            # Face (~8 points)
            if results.face_landmarks:
                for idx in FACE_INDICES_TO_KEEP:
                    if idx < len(results.face_landmarks.landmark):
                        lm = results.face_landmarks.landmark[idx]
                        node_features.append(lm_to_xyzc(lm, default_conf=1.0))
                    else:
                        node_features.append([0.0, 0.0, 0.0, 0.0])
            else:
                for _ in range(len(FACE_INDICES_TO_KEEP)):
                    node_features.append([0.0, 0.0, 0.0, 0.0])

            # --- Calculate Bone Vectors ---
            # Bone vectors represent the direction and length of a body segment.
            # We'll calculate a few key vectors that capture significant movements.
            # A vector from point A to B is B_coords - A_coords.
            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark
                # Right Arm
                for start, end in [(mp_holistic.PoseLandmark.RIGHT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_ELBOW),
                                   (mp_holistic.PoseLandmark.RIGHT_ELBOW, mp_holistic.PoseLandmark.RIGHT_WRIST)]:
                    vec = [lm[end].x - lm[start].x, lm[end].y - lm[start].y, lm[end].z - lm[start].z]
                    frame_bone_vectors.extend(vec)
                # Left Arm
                for start, end in [(mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.LEFT_ELBOW),
                                   (mp_holistic.PoseLandmark.LEFT_ELBOW, mp_holistic.PoseLandmark.LEFT_WRIST)]:
                    vec = [lm[end].x - lm[start].x, lm[end].y - lm[start].y, lm[end].z - lm[start].z]
                    frame_bone_vectors.extend(vec)
            else:
                # Pad with zeros if landmarks are not detected
                frame_bone_vectors = [0.0] * 12 # 4 bones * 3 axes
            
            frames.append(np.array(node_features, dtype=np.float32))
            torso_lengths.append(frame_torso_length)
            bone_vectors_per_frame.append(frame_bone_vectors)

    finally:
        cap.release()

    if not frames:
        return None, None, None
    
    # Stack the arrays from the list of frames
    landmark_seq = np.stack(frames, axis=0)
    torso_seq = np.array(torso_lengths, dtype=np.float32)
    bone_seq = np.array(bone_vectors_per_frame, dtype=np.float32)
    
    return landmark_seq, torso_seq, bone_seq

def main():
    with open(config["SPLIT_FILE_PATH"], 'r') as f:
        data = json.load(f)
    video_ids = list(data.keys())
    video_paths = [(vid, os.path.join(config["VIDEO_SOURCE_DIR"], f"{vid}.mp4")) for vid in video_ids]

    node_labels = []
    for p in config["POSE_NODES"]:
        node_labels.append(('pose', p.name))
    node_labels.append(('pose_mid', 'mid_shoulder'))
    node_labels.append(('pose_mid', 'mid_hip'))
    for i in range(21):
        node_labels.append(('hand', f"L_{i}"))
    for i in range(21):
        node_labels.append(('hand', f"R_{i}"))
    for idx in FACE_INDICES_TO_KEEP:
        node_labels.append(('face', f"F_{idx}"))

    adjacency = build_adjacency_map(node_labels)

    holistic = mp_holistic.Holistic(static_image_mode=False,
                                    model_complexity=1,
                                    min_detection_confidence=0.5,
                                    min_tracking_confidence=0.5)

    extracted_landmarks = {}
    extracted_torsos = {}
    extracted_bones = {}

    try:
        for i, (vid, path) in enumerate(tqdm(video_paths, desc="Videos")):
            if not os.path.exists(path):
                continue
            seq, torso_len, bone_vec = extract_compact_landmarks(path, holistic)
            if seq is not None:
                extracted_landmarks[vid] = seq
                extracted_torsos[vid] = torso_len
                extracted_bones[vid] = bone_vec
            if (i > 0) and (i+1) % config["SAVE_CHECKPOINT_EVERY_N_VIDEOS"] == 0:
                print(f"Saving checkpoint with {len(extracted_landmarks)} videos")
                np.savez_compressed(config["OUTPUT_NPZ_PATH"],
                                     _node_labels=np.array(node_labels, dtype=object),
                                     _adjacency=adjacency,
                                     _torso_lengths=extracted_torsos,
                                     _bone_vectors=extracted_bones,
                                     **extracted_landmarks)
    finally:
        holistic.close()

    np.savez_compressed(config["OUTPUT_NPZ_PATH"],
                        _node_labels=np.array(node_labels, dtype=object),
                        _adjacency=adjacency,
                        _torso_lengths=extracted_torsos,
                        _bone_vectors=extracted_bones,
                        **extracted_landmarks)
    print("Saved:", config["OUTPUT_NPZ_PATH"])

if __name__ == "__main__":
    main()


Videos:   5%|▍         | 249/5118 [18:26<5:16:31,  3.90s/it]

Saving checkpoint with 250 videos


Videos:  10%|▉         | 499/5118 [1:17:22<2:45:26,  2.15s/it]   

Saving checkpoint with 500 videos


Videos:  15%|█▍        | 749/5118 [1:29:24<4:05:33,  3.37s/it]

...

Saving checkpoint with 4750 videos


Videos:  98%|█████████▊| 4999/5118 [6:23:52<07:09,  3.61s/it]

Saving checkpoint with 5000 videos


Videos: 100%|██████████| 5118/5118 [6:30:59<00:00,  4.58s/it]


Saved: D:\Landmarks\Bones+Joints\Landmarks_GCN_augmented.npz
